<a href="https://colab.research.google.com/github/Gfernandezv/GPXsync/blob/main/GPX_sync.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalación de las dependencias.

In [ ]:
!pip install gpxpy
!pip install folium
!pip install ipywidgets
!pip install ipyleaflet
!apt-get install -y ffmpeg

cargar archivo de mapa

In [ ]:
from google.colab import files

uploaded = files.upload()
GPX_name = list(uploaded.keys())[0]
GPX_content = uploaded[GPX_name]
gpx_file_path = GPX_name


Comenzar con el analisis, cargar el mapa

In [ ]:
import gpxpy
from ipyleaflet import Map, CircleMarker, basemaps, FullScreenControl
from ipywidgets import IntSlider, VBox, Layout, HTML
from IPython.display import display

gpx_content = GPX_content.decode('utf-8')
gpx = gpxpy.parse(gpx_content)

# Leer las coordenadas de cada punto
coords = [(point.latitude, point.longitude) for point in gpx.tracks[0].segments[0].points]
timestamps = [point.time for point in gpx.tracks[0].segments[0].points]

# mapa con ipyleaflet
m = Map(center=coords[0], zoom=16, basemap=basemaps.Esri.WorldStreetMap, controls=[FullScreenControl()])

# Crear marcadores y añadirlos al mapa
markers = [CircleMarker(location=coord, color="red", fill_color="red", radius=2) for coord in coords]
for marker in markers:
    marker.visible = False
    m.add_layer(marker)

# Crear widget slider
slider_layout = Layout(width='100%')
slider = IntSlider(min=0, max=len(coords) - 1, step=1, value=0, description='Punto', layout=slider_layout)

# Widget HTML para mostrar información del punto
info_widget = HTML()

# Función para actualizar el mapa y la información del punto
def update_map_and_info(change):
    index = change['new']
    for i, marker in enumerate(markers):
        marker.visible = i == index
        marker.color = "blue" if i == index else "red"
    m.center = markers[index].location

    # Mostrar información del punto seleccionado
    timestamp = timestamps[index]
    info_widget.value = f'<b>Fecha y Hora:</b> {timestamp.strftime("%Y-%m-%d %H:%M:%S")}<br><b>Coordenadas:</b> {markers[index].location}'

# Asignar la función al evento de cambio del slider
slider.observe(update_map_and_info, 'value')

# Mostrar el mapa, el slider y la información del punto
display(VBox([m, slider, info_widget]))



Selecciona el punto de inicio del video utilizando la barra.
Carga el video en google drive y modifica la linea de file_path.
Luego ejecutar el codigo de la celda.

In [ ]:
import os

# Ruta al archivo en Colab
file_path = '/content/video1.mp4'

# Obtener la fecha y hora del punto seleccionado a través del slider
index = slider.value
timestamp = timestamps[index]

# Obtener la marca de tiempo en segundos desde la época
timestamp_seconds = timestamp.timestamp()

# Establecer la marca de tiempo de modificación del archivo
os.utime(file_path, (timestamp_seconds, timestamp_seconds))


Ejecutar la siguiente celda para confirmar que la fecha de modificación fue cambiada.

In [ ]:
import os

# Ruta al archivo en Colab
file_path = '/content/video1.mp4'

# Obtener la fecha de modificación del archivo
fecha_modificacion_timestamp = os.path.getmtime(file_path)

# Convertir la marca de tiempo a un objeto datetime
fecha_modificacion_datetime = datetime.fromtimestamp(fecha_modificacion_timestamp)

# Imprimir la fecha de modificación
print(f'Fecha de modificación del archivo: {fecha_modificacion_datetime.strftime("%Y-%m-%d %H:%M:%S")}')



Fecha de modificación del archivo: 2023-12-11 22:02:50
